In [1]:
from skimage.feature import local_binary_pattern
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

def calcula_lbp(imagem, P, R, metodo):
    # Calcular o LBP
    lbp = local_binary_pattern(imagem, P, R, metodo)
    # Calcular o histograma LBP
    n_bins = int(lbp.max() + 1)
    hist, bins = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins), density=True)
    return hist

# Abrir o arquivo para escrita
arquivo_resultados = open('resultados.txt', 'w')

# Parâmetros LBP
P = 8  # Número de pontos vizinhos
R = 2  # Raio
metodo = 'nri_uniform'

# Diretórios
diretorios = ['harmonico/fold1', 'harmonico/fold2', 'harmonico/fold3']

hist = []
tags = []

# Carregar as imagens e calcular os histogramas LBP
for i, diretorio in enumerate(diretorios):
    lista_arquivos = os.listdir(diretorio)
    for j, arquivo_img in enumerate(lista_arquivos):
        caminho_imagem = os.path.join(diretorio, arquivo_img)
        imagem = cv2.imread(caminho_imagem, cv2.IMREAD_GRAYSCALE)
        hist.append(calcula_lbp(imagem, P, R, metodo))
        tags.append(j // 30 + 1)  # Tag para cada imagem

hist = np.array(hist)
tags = np.array(tags)

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular as métricas
        y_pred = best_model.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem
    
    mean_precision = np.mean(precisions) * 100
    std_precision = np.std(precisions) * 100
    
    mean_recall = np.mean(recalls) * 100
    std_recall = np.std(recalls) * 100
    
    mean_f1_score = np.mean(f1_scores) * 100
    std_f1_score = np.std(f1_scores) * 100

    return (mean_accuracy, std_accuracy, 
            mean_precision, std_precision, 
            mean_recall, std_recall, 
            mean_f1_score, std_f1_score)

# Realizar a validação cruzada
(mean_accuracy, std_accuracy, 
 mean_precision, std_precision, 
 mean_recall, std_recall, 
 mean_f1_score, std_f1_score) = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}% (Desvio padrão: {std_accuracy:.2f}%)')
print(f'Precisão média: {mean_precision:.2f}% (Desvio padrão: {std_precision:.2f}%)')
print(f'Recall médio: {mean_recall:.2f}% (Desvio padrão: {std_recall:.2f}%)')
print(f'F1-score médio: {mean_f1_score:.2f}% (Desvio padrão: {std_f1_score:.2f}%)')


Acurácia média: 74.44% (Desvio padrão: 3.50%)
Precisão média: 75.56% (Desvio padrão: 3.86%)
Recall médio: 74.44% (Desvio padrão: 3.50%)
F1-score médio: 74.16% (Desvio padrão: 3.40%)


In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Função para realizar a validação cruzada com KNN
def kfoldcv_knn(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])

        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]

        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Definir a grade de parâmetros para o Grid Search do KNN
        param_grid = {
            'n_neighbors': [3, 5, 7, 9, 11, 13],
            'metric': ['canberra', 'minkowski', 'euclidean', 'manhattan']
        }

        # Treinar o modelo KNN usando Grid Search
        knn_model = KNeighborsClassifier()
        grid_search = GridSearchCV(knn_model, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)

        # Melhor modelo encontrado pelo Grid Search
        best_knn = grid_search.best_estimator_

        # Fazer previsões e calcular as métricas
        y_pred = best_knn.predict(X_test)
        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, average='weighted'))
        recalls.append(recall_score(y_test, y_pred, average='weighted'))
        f1_scores.append(f1_score(y_test, y_pred, average='weighted'))

    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem
    
    mean_precision = np.mean(precisions) * 100
    std_precision = np.std(precisions) * 100
    
    mean_recall = np.mean(recalls) * 100
    std_recall = np.std(recalls) * 100
    
    mean_f1_score = np.mean(f1_scores) * 100
    std_f1_score = np.std(f1_scores) * 100

    return mean_accuracy, std_accuracy, mean_precision, std_precision, mean_recall, std_recall, mean_f1_score, std_f1_score

# Realizar a validação cruzada com KNN
mean_accuracy, std_accuracy, mean_precision, std_precision, mean_recall, std_recall, mean_f1_score, std_f1_score = kfoldcv_knn(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}%')
print(f'Precisão média: {mean_precision:.2f}%')
print(f'Desvio padrão da precisão: {std_precision:.2f}%')
print(f'Recall médio: {mean_recall:.2f}%')
print(f'Desvio padrão do recall: {std_recall:.2f}%')
print(f'F1-score médio: {mean_f1_score:.2f}%')
print(f'Desvio padrão do F1-score: {std_f1_score:.2f}%')


Acurácia média: 60.44%
Desvio padrão da acurácia: 1.81%
Precisão média: 60.27%
Desvio padrão da precisão: 1.76%
Recall médio: 60.44%
Desvio padrão do recall: 1.81%
F1-score médio: 59.49%
Desvio padrão do F1-score: 2.05%


In [2]:
# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    
    accuracies = []
    
    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Treinar o modelo SVM
        svm_model = SVC(kernel='linear', C=0.20, random_state=42)
        svm_model.fit(X_train, y_train)
        
        # Fazer previsões e calcular a acurácia
        y_pred = svm_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}')


Acurácia média: 73.33
Desvio padrão da acurácia: 2.60


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [0.289],
            'gamma': [1],
            'kernel': ['linear', 'rbf']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular a acurácia
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}%')


Acurácia média: 74.44%
Desvio padrão da acurácia: 3.50%


In [5]:
# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Configurar os parâmetros para o GridSearchCV (SVM)
        param_grid = {
            'C': [50, 10, 1, 0.1, 0.001, 0.0001],
            'gamma': [100, 50, 10, 1, 0.1, 0.50, 0.001, 0.0001],
            'kernel': ['rbf']
        }

        svm_model = SVC(random_state=42)
        grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        # Melhor modelo encontrado pelo Grid Search
        best_model = grid_search.best_estimator_

        # Fazer previsões e calcular a acurácia
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}%')


Acurácia média: 71.11%
Desvio padrão da acurácia: 1.29%


In [6]:
from sklearn.ensemble import RandomForestClassifier

# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    
    accuracies = []
    
    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        # Treinar o modelo SVM
        rf_model = RandomForestClassifier(n_estimators=4000, random_state=0, n_jobs=-1)
        rf_model.fit(X_train, y_train)
         
        # Fazer previsões e calcular a acurácia
        y_pred = rf_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}')

Acurácia média: 65.22
Desvio padrão da acurácia: 2.62


In [8]:
# Função para realizar a validação cruzada
def kfoldcv(hist, tags, k):
    fold_size = len(hist) // k
    accuracies = []
    
    for i in range(k):
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])
        
        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]
        
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        
        param_grid = {
            'n_estimators': [200, 2000, 4000],
            'max_depth': [None, 10, 20, 30, 40, 50]
        }
        rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
        grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)
    
    mean_accuracy = np.mean(accuracies) * 100
    std_accuracy = np.std(accuracies) * 100

    return mean_accuracy, std_accuracy


# Realizar a validação cruzada
mean_accuracy, std_accuracy = kfoldcv(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}')

Acurácia média: 64.56
Desvio padrão da acurácia: 2.59


In [3]:
from sklearn.neighbors import KNeighborsClassifier

# Função para realizar a validação cruzada com KNN
def kfoldcv_knn(hist, tags, k):
    # Assumindo que os dados já estão ordenados conforme os folds nos diretórios
    fold_size = len(hist) // k
    accuracies = []

    for i in range(k):
        # Define os índices de teste e treino baseados na ordem
        test_idx = np.arange(i*fold_size, (i+1)*fold_size)
        train_idx = np.concatenate([np.arange(0, i*fold_size), np.arange((i+1)*fold_size, len(hist))])

        X_train, X_test = hist[train_idx], hist[test_idx]
        y_train, y_test = tags[train_idx], tags[test_idx]

        # Padronizar os dados
        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Definir a grade de parâmetros para o Grid Search
        param_grid = {
            'n_neighbors': [3, 5, 7, 9, 11],
            'metric': ['canberra', 'minkowski', 'euclidean', 'manhattan']
        }

        # Treinar o modelo KNN usando Grid Search
        knn_model = KNeighborsClassifier()
        grid_search = GridSearchCV(knn_model, param_grid, cv=3, scoring='accuracy')
        grid_search.fit(X_train, y_train)

        # Melhor modelo encontrado pelo Grid Search
        best_knn = grid_search.best_estimator_

        # Fazer previsões e calcular a acurácia
        y_pred = best_knn.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies.append(accuracy)

    mean_accuracy = np.mean(accuracies) * 100  # Converter para porcentagem
    std_accuracy = np.std(accuracies) * 100  # Converter para porcentagem

    return mean_accuracy, std_accuracy

# Realizar a validação cruzada com KNN
mean_accuracy, std_accuracy = kfoldcv_knn(hist, tags, 3)

print(f'Acurácia média: {mean_accuracy:.2f}%')
print(f'Desvio padrão da acurácia: {std_accuracy:.2f}%')

Acurácia média: 60.44%
Desvio padrão da acurácia: 1.81%
